In [1]:
import os, sys
import tensorflow as tf
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

In [2]:
# context manager to suppress stdoutm for annoying long printouts
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [3]:
experiment = 'tf_resnet50'
batch_size = 300

In [4]:
platform = 'paperspace' #@param ['colab', 'paperspace'] {allow-input: true}

if platform == 'jupyter_notebook':
    challenge_data_dir = '../data/algonauts_2023_challenge_data'
    exp_output_dir = f'../data/out/{experiment}'

if platform == 'paperspace':
    challenge_data_dir = '/storage/algonauts_2023_challenge_data'
    exp_output_dir = f'/notebooks/data/out/{experiment}'

## Functions not aligned with the current structuree

In [5]:
def train_pca(model, train_dataset):
    
    features = []
    for batch in train_dataset:
        # Extract features
        with HiddenPrints():
            ft = model.predict(batch)
        # Flatten the features
        ft = ft.reshape(ft.shape[0], -1)
        # Append to list
        features.append(ft)
    # Combine features from all batches
    del model
    features = np.vstack(features)
    # Fit PCA to combined features
    pca = PCA(n_components=100)
    pca.fit(features)
    # print('plot the explained variance')
    # plt.plot(pca.explained_variance_ratio_)
    # plt.show()
    # print('plot the cumulative explained variance')
    # plt.plot(np.cumsum(pca.explained_variance_ratio_))
    # plt.show()
    return pca

def extract_and_transform_features(dataset, model, pca):
    features = []
    for batch in dataset:
        with HiddenPrints():
            ft = model.predict(batch)
        # Flatten the features
        ft = ft.reshape(ft.shape[0], -1)
        # Fit PCA to batch of features
        ft = pca.transform(ft)
        features.append(ft)
    return np.vstack(features)

def load_datasets(dataset, image_size, normalise_dataset):
    train_val_imgs_paths = [os.path.join(dataset.train_img_dir, img_name) for img_name in dataset.training_img_list]
    train_paths = [train_val_imgs_paths[i] for i in dataset.idxs_train]
    val_paths = [train_val_imgs_paths[i] for i in dataset.idxs_val]
    test_imgs_paths = [os.path.join(dataset.test_img_dir, img_name) for img_name in dataset.test_img_list]

    train_ds = tf.data.Dataset.from_tensor_slices(train_paths)
    val_ds = tf.data.Dataset.from_tensor_slices(val_paths)
    test_ds = tf.data.Dataset.from_tensor_slices(test_imgs_paths)

    def load_image(image_path):
        # load image file
        img = tf.io.read_file(image_path)
        # decode image
        img = tf.image.decode_jpeg(img, channels=3)
        # resize image to a fixed shape
        img = tf.image.resize(img, image_size)
        return img
    
    def process(ds):
        ds = ds.map(load_image)
        ds = ds.map(normalise_dataset)
        ds = ds.batch(batch_size)
        ds = ds.prefetch(tf.data.AUTOTUNE)
        return ds
    
    train_ds = process(train_ds)
    val_ds = process(val_ds)
    test_ds = process(test_ds)

    return train_ds, val_ds, test_ds

def save_predictions(lh_fmri_test_pred, rh_fmri_test_pred, subject_submission_dir):
    lh_fmri_test_pred = lh_fmri_test_pred.astype(np.float32)
    rh_fmri_test_pred = rh_fmri_test_pred.astype(np.float32)

    np.save(os.path.join(subject_submission_dir, 'lh_pred_test.npy'), lh_fmri_test_pred)
    np.save(os.path.join(subject_submission_dir, 'rh_pred_test.npy'), rh_fmri_test_pred)

## Specify model and appropriate image preprocessing steps
Define a function to load and return the model, printing its layers.
Due to memory constraints, we will delete the model from memory in each loop. Here we first load it, print its
nodes and delete it.

In [6]:
def load_model():
    # Take models from here https://keras.io/api/applications/
    model = tf.keras.applications.EfficientNetB4(weights='imagenet', include_top=False)
    return model

normalise_dataset = tf.keras.applications.efficientnet.preprocess_input
image_size = (300, 300)

model = load_model()
print(*(layer.name for layer in model.layers), sep=' -> ')
del model

71686520/71686520 [==============================] - 1s 0us/step
input_1 -> rescaling -> normalization -> tf.math.truediv -> stem_conv_pad -> stem_conv -> stem_bn -> stem_activation -> block1a_dwconv -> block1a_bn -> block1a_activation -> block1a_se_squeeze -> block1a_se_reshape -> block1a_se_reduce -> block1a_se_expand -> block1a_se_excite -> block1a_project_conv -> block1a_project_bn -> block1b_dwconv -> block1b_bn -> block1b_activation -> block1b_se_squeeze -> block1b_se_reshape -> block1b_se_reduce -> block1b_se_expand -> block1b_se_excite -> block1b_project_conv -> block1b_project_bn -> block1b_drop -> block1b_add -> block2a_expand_conv -> block2a_expand_bn -> block2a_expand_activation -> block2a_dwconv_pad -> block2a_dwconv -> block2a_bn -> block2a_activation -> block2a_se_squeeze -> block2a_se_reshape -> block2a_se_reduce -> block2a_se_expand -> block2a_se_excite -> block2a_project_conv -> block2a_project_bn -> block2b_expand_conv -> block2b_expand_bn -> block2b_expand_activatio

## Select layers and subjects
Now let's define which layer(s) we will pick from, and which subject(s) to process

In [7]:
layers = ['block6d_bn']
subjects = [1, 2, 3, 4, 5, 6, 7, 8]


## Run pipeline

In [8]:
from src.algonauts.evaluators import correlations as corr
from src.algonauts.data_processors.nsd_dataset import NSDDataset

for layer_name in layers:
    print(f'Running for layer {layer_name}')
    for subj in subjects:
        print(f'Running for subject {subj}')

        # Set data directories based on parameters
        output_dir = f'{exp_output_dir}/{layer_name}'
        dataset = NSDDataset(challenge_data_dir, output_dir, subj)

        print('Loading datasets...')
        train_ds, val_ds, test_ds = load_datasets(dataset, image_size, normalise_dataset)
        print('Datasets loaded')

        # Load model for the iteration
        model = load_model()
        sliced_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

        # Train PCA
        print('Training PCA...')
        pca = train_pca(sliced_model, train_ds)
        print('PCA over')

        # Extract and transform features
        print('Extracting and transforming features...')
        train_features = extract_and_transform_features(train_ds, sliced_model, pca)
        val_features = extract_and_transform_features(val_ds, sliced_model, pca)
        test_features = extract_and_transform_features(test_ds, sliced_model, pca)
        print('Features extracted and transformed')

        # Delete model to free up memory
        del model, pca

        # Fit regression
        print('Fitting regression...')
        reg_lh = LinearRegression().fit(train_features, dataset.lh_fmri_train)
        reg_rh = LinearRegression().fit(train_features, dataset.rh_fmri_train)
        print('Regression fitted')
        
        # Use fitted linear regressions to predict the validation and test fMRI data
        print('Predicting fMRI data...')
        lh_fmri_val_pred = reg_lh.predict(val_features)
        lh_fmri_test_pred = reg_lh.predict(test_features)
        rh_fmri_val_pred = reg_rh.predict(val_features)
        rh_fmri_test_pred = reg_rh.predict(test_features)
        print('fMRI data predicted')
        # Calculate correlations for each hemispher
        print('Calculating correlations...')
        lh_correlation = corr.calculate_correlation(lh_fmri_val_pred, dataset.lh_fmri_val)
        rh_correlation = corr.calculate_correlation(rh_fmri_val_pred, dataset.rh_fmri_val)  
        print('Correlations calculated')
    
        corr.plot_and_write_correlations(dataset, lh_correlation, rh_correlation, exp_output_dir, layer_name, subj)

Running for layer block6d_bn
Running for subject 1
Training images: 9841
Test images: 159
LH training fMRI data shape:
(9841, 19004)
(Training stimulus images × LH vertices)

RH training fMRI data shape:
(9841, 20544)
(Training stimulus images × RH vertices)
Training images: 9841
Test images: 159
Training stimulus images: 8856

Validation stimulus images: 985

Test stimulus images: 159
Loading datasets...
Datasets loaded
Training PCA...
PCA over
Extracting and transforming features...
Features extracted and transformed
Fitting regression...
